In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io as io
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

In [2]:
def labelprocessing(label_trainpath,label_testpath,glass_label_path):
    
    raw_data_train=pd.read_csv(label_trainpath,names=["mixed_information_train"])
    data_unmixed_train = raw_data_train["mixed_information_train"].str.split("\t",expand=True)
    data_unmixed_train.columns=['index','eye_color','face_shape','file_name']
    data_unmixed_train.drop(columns=['index','face_shape','file_name'],inplace=True)
    data_unmixed_train.drop(index=0,inplace=True)
    
    raw_data_test=pd.read_csv(label_testpath,names=["mixed_information_test"])
    data_unmixed_test = raw_data_test["mixed_information_test"].str.split("\t",expand=True)
    data_unmixed_test.columns=['index','eye_color','face_shape','file_name']
    data_unmixed_test.drop(columns=['index','face_shape','file_name'],inplace=True)
    data_unmixed_test.drop(index=0,inplace=True)

    raw_data_newlabel=pd.read_csv(glass_label_path,names=["sunglass"])
    #raw_data_newlabel.head()
    
    YTrain=data_unmixed_train
    YTrain_newlabel=raw_data_newlabel
    YTest=data_unmixed_test
    YTrain=YTrain.values
    YTrain_newlabel=YTrain_newlabel.values
    YTest=YTest.values
    YTrain=np.array([int(y) for y in YTrain])
    YTrain_newlabel=np.array([int(y) for y in YTrain_newlabel])
    YTest=np.array([int(y) for y in YTest])
    YTrain=YTrain.reshape(len(YTrain),1)
    YTrain_newlabel=YTrain_newlabel.reshape(len(YTrain_newlabel),1)
    YTest=YTest.reshape(len(YTest),1)
    
    return YTrain,YTest,YTrain_newlabel

def change_shape(img):
    [NUM,row,col,dimension]=img.shape
    img=img.reshape(NUM,row*col*dimension)
    return img

def png_to_rgb(image):
    image = image[:, :, :3]
    image = image.astype(float)
    return image

def datapreprocessing(data_train_path,data_test_path):
    XTrain=[]
    XTrain_newlabel_raw=[]
    XTrain_newlabel=[]
    XTrain_predict=[]
    XTest_temp=[]
    XTest=[]
    coll_train = io.ImageCollection(data_train_path)
    coll_test = io.ImageCollection(data_test_path)
    for img in coll_train:
        intern_train=cv2.resize(img, (120,120))
        intern_train=np.array(intern_train)
        XTrain.append(intern_train) 
    XTrain=np.array(XTrain)/255
    for img in coll_test:
        intern_test=cv2.resize(img, (120,120))
        intern_test=np.array(intern_test)
        XTest_temp.append(intern_test)
    XTest_temp=np.array(XTest_temp)/255
    XTrain_newlabel_raw=XTrain[:500,:,:,:]
    for (i,value) in enumerate(XTrain_newlabel_raw):
        XTrain_newlabel.append(value.flatten())
    for (i,value) in enumerate(XTrain):
        XTrain_predict.append(value.flatten()) 
    for (i,value) in enumerate(XTest_temp):
        XTest.append(value.flatten())
    XTrain_newlabel=np.array(XTrain_newlabel)
    XTrain_predict=np.array(XTrain_predict)
    return XTrain,XTest,XTrain_newlabel,XTrain_predict

def crossvalidation_judgeglass(XTrain_newlabel,YTrain_newlabel):
    rf=RandomForestClassifier(max_depth=25,n_estimators=70)
    rf.fit(XTrain_newlabel,YTrain_newlabel)  
    return rf

def creatlable_judgeglass(XTrain_predict,rf,YTrain):
    YTrain_glass=[]
    YTrain_noglass=[]
    XTrain_glass=[]
    XTrain_noglass=[]
    creat_label=rf.predict(XTrain_predict)
    creat_label=np.array(creat_label)
    for i in range (len(creat_label)):
        if creat_label[i]==1:
            XTrain_glass.append(XTrain_predict[i])
            YTrain_glass.append(YTrain[i])
        elif creat_label[i]==0:
            XTrain_noglass.append(XTrain_predict[i])
            YTrain_noglass.append(YTrain[i])
    XTrain_glass=np.array(XTrain_glass)
    XTrain_glass=XTrain_glass.reshape(len(XTrain_glass),120,120,4)
    XTrain_noglass=np.array(XTrain_noglass)
    XTrain_noglass=XTrain_noglass.reshape(len(XTrain_noglass),120,120,4)
    YTrain_glass=np.array(YTrain_glass)
    YTrain_noglass=np.array(YTrain_noglass)
    return XTrain_noglass,XTrain_glass,YTrain_noglass,YTrain_glass

def predict_withnoglass(XTrain_noglass,YTrain_noglass):
    import time
    XTrain_noglass=change_shape(XTrain_noglass)
    pca=PCA(n_components=200,svd_solver='randomized',whiten=True).fit(XTrain_noglass)
    XTrain_noglass = pca.transform(XTrain_noglass)
    start=time.perf_counter()
    fully_connected_layer_num=[3]
    unit_count=[200]
    for fully_connected_layer in fully_connected_layer_num:
        for unit in unit_count:
            model=Sequential()
            for i in range(fully_connected_layer):
                model.add(Dense(unit,activation='relu'))
            #output layer
            model.add(Dense(5,activation='sigmoid'))
            #compile
            model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
            #fit
            model.fit(XTrain_noglass,YTrain_noglass,batch_size=64,epochs=5)
    end=time.perf_counter()
    time=end-start
    return model,pca,time

def acc_Test(model,XTest_noglass,YTest_noglass):
    Yres=[]
    res=model.predict(XTest_noglass)
    for i in range(len(res)):
        Yres.append(np.argmax(res[i]))
    Yres=np.array(Yres)
    acc=np.mean(Yres==YTest_noglass.flatten())
    return acc

def main(label_trainpath,label_testpath,glass_label_path,data_train_path,data_test_path):
    YTrain,YTest,YTrain_newlabel=labelprocessing(label_trainpath,label_testpath,glass_label_path)
    XTrain,XTest,XTrain_newlabel,XTrain_predict=datapreprocessing(data_train_path,data_test_path)
    rf=crossvalidation_judgeglass(XTrain_newlabel,YTrain_newlabel)
    XTrain_noglass,XTrain_glass,YTrain_noglass,YTrain_glass=creatlable_judgeglass(XTrain_predict,rf,YTrain)
    XTest_noglass,XTest_glass,YTest_noglass,YTest_glass=creatlable_judgeglass(XTest,rf,YTest)
    model,pca,time=predict_withnoglass(XTrain_noglass,YTrain_noglass)
    XTest_noglass=change_shape(XTest_noglass)
    XTest_noglass = pca.transform(XTest_noglass)
    acc=acc_Test(model,XTest_noglass,YTest_noglass)
    return acc,time

In [1]:
label_trainpath=r".\Datasets\cartoon_set\labels.csv"
label_testpath=r".\Datasets\cartoon_set_test\labels.csv"
data_train_path=r".\Datasets\cartoon_set\img\*.png"
data_test_path= r".\Datasets\cartoon_set_test\img\*.png"
glass_label_path=r".\B2\Semi_supervised_learning_labels.csv"

In [4]:
acc,time=main(label_trainpath,label_testpath,glass_label_path,data_train_path,data_test_path)
print("TaskB2: Accuracy of Fully Connected Neural Network is {}".format(acc))

C:\Users\fdl\AppData\Local\Temp\ipykernel_17332\3513425238.py:79: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(XTrain_newlabel,YTrain_newlabel)


Epoch 1/5
128/128 [==============================] - 1s 1ms/step - loss: 0.7576 - accuracy: 0.6987
Epoch 2/5
128/128 [==============================] - 0s 1ms/step - loss: 0.1457 - accuracy: 0.9498
Epoch 3/5
128/128 [==============================] - 0s 1ms/step - loss: 0.0512 - accuracy: 0.9844
Epoch 4/5
128/128 [==============================] - 0s 1ms/step - loss: 0.0236 - accuracy: 0.9940
Epoch 5/5
64/64 [==============================] - 0s 782us/step


0.9488440727988194